## Livro para consulta:
- https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html
- https://jakevdp.github.io/PythonDataScienceHandbook/03.09-pivot-tables.html
    

# 0. Índice <a name="Contents"></a>
1. [Importando bibliotecas](#import)
2. [Carregando o dataframe](#read)
5. [GroupBy: Dividir, Aplicar, Combinar (Split, Apply, Combine)](#split)
5. [Estrutura do Groupby](#estrutura)
5. [Pivot table](#pivot)
5. [Group by DTNASC e contagem da IDADEMAE](#dtnasc)
    1. [Em SQL](#sql)
    1. [Em Excel](#excel)
    1. [Python com ``groupby()``](#py_group)
    1. [Python com ``pivot_table()``](#py_pivot)



# 1. Importando bibliotecas <a name="import"></a>

<div style="text-align: right"
     
[Voltar ao índice](#Contents)

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme() 
%matplotlib inline

# 2. Carregando o dataframe SINASC <a name="read"></a>
<div style="text-align: right"
     
[Voltar ao índice](#Contents)

SINASC - Sistema de Informações de Nascidos Vivos

     
O DATASUS desenvolveu o Sistema de Informações sobre Nascidos Vivos (SINASC) visando reunir informações epidemiológicas referentes aos nascimentos informados em todo território nacional. Sua implantação ocorreu de forma lenta e gradual em todas as Unidades da Federação.     
     
http://www2.datasus.gov.br/DATASUS/index.php?area=060702


In [3]:
sinasc_raw = pd.read_csv('SINASC_RO_2019.csv')
sinasc_raw.head()

,ORIGEM,CODESTAB,CODMUNNASC,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,CODOCUPMAE,QTDFILVIVO,QTDFILMORT,...,KOTELCHUCK,CONTADOR,munResStatus,munResTipo,munResNome,munResUf,munResLat,munResLon,munResAlt,munResArea
0,1,2679477.0,110001,1,19,5.0,8 a 11 anos,NaN,0.0,0.0,...,5,1,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
1,1,2679477.0,110001,1,29,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,2,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
2,1,2679477.0,110001,1,37,9.0,8 a 11 anos,513205.0,2.0,0.0,...,5,3,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
3,1,2516500.0,110001,1,30,5.0,12 anos ou mais,231205.0,0.0,0.0,...,4,4,ATIVO,MUNIC,Alto Alegre dos Parecis,Rondônia,-12.13178,-61.85308,397.0,3958.273
4,1,2516500.0,110001,1,30,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,5,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025


In [4]:
sinasc_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27028 entries, 0 to 27027
Data columns (total 69 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ORIGEM        27028 non-null  int64  
 1   CODESTAB      26913 non-null  float64
 2   CODMUNNASC    27028 non-null  int64  
 3   LOCNASC       27028 non-null  int64  
 4   IDADEMAE      27028 non-null  int64  
 5   ESTCIVMAE     26711 non-null  float64
 6   ESCMAE        26716 non-null  object 
 7   CODOCUPMAE    24121 non-null  float64
 8   QTDFILVIVO    25455 non-null  float64
 9   QTDFILMORT    24930 non-null  float64
 10  CODMUNRES     27028 non-null  int64  
 11  GESTACAO      25796 non-null  object 
 12  GRAVIDEZ      26949 non-null  object 
 13  PARTO         26979 non-null  object 
 14  CONSULTAS     27028 non-null  int64  
 15  DTNASC        27028 non-null  object 
 16  HORANASC      27007 non-null  float64
 17  SEXO          27024 non-null  object 
 18  APGAR1        26932 non-nu

# 3.0 GroupBy: Dividir, Aplicar, Combinar (Split, Apply, Combine)<a name="split"></a>
<div style="text-align: right"
     
[Voltar ao índice](#Contents)



<img src="split-apply-combine.png" width="750" align="center">


https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html

In [5]:
sinasc_1 = sinasc_raw[['ESCMAE', 'IDADEMAE','DTNASC']]
sinasc_1.head()

,ESCMAE,IDADEMAE,DTNASC
0,8 a 11 anos,19,2019-02-19
1,8 a 11 anos,29,2019-02-21
2,8 a 11 anos,37,2019-02-25
3,12 anos ou mais,30,2019-03-20
4,8 a 11 anos,30,2019-03-23


In [7]:
sinasc_1.mean()

TypeError: can only concatenate str (not "int") to str

In [8]:
sinasc_1.ESCMAE.value_counts()

ESCMAE
8 a 11 anos        15633
12 anos ou mais     5767
4 a 7 anos          4760
1 a 3 anos           516
Nenhuma               40
Name: count, dtype: int64

# 4.0  Estrutura do Groupby <a name="estrutura"></a>
<div style="text-align: right"
     
[Voltar ao índice](#Contents)



In [9]:
sinasc_1.groupby('ESCMAE')

In [10]:
sinasc_1.groupby('ESCMAE').mean()

TypeError: Could not convert 2019-03-272019-03-112019-03-282019-01-102019-01-242019-01-082019-01-142019-01-122019-01-232019-01-102019-01-152019-01-192019-01-242019-01-312019-02-152019-02-212019-02-112019-02-032019-02-142019-02-162019-02-162019-02-262019-02-232019-03-012019-03-012019-03-072019-03-152019-03-062019-03-032019-03-042019-03-182019-03-232019-03-262019-03-232019-03-262019-04-152019-04-082019-04-122019-04-232019-04-022019-04-082019-04-082019-04-122019-04-122019-04-192019-04-202019-04-222019-05-152019-05-082019-05-082019-05-102019-05-112019-05-272019-05-282019-06-172019-06-062019-06-172019-06-182019-06-292019-02-262019-03-272019-05-022019-05-092019-05-182019-01-022019-02-272019-06-202019-01-042019-02-182019-03-032019-04-112019-05-282019-05-312019-01-012019-02-132019-02-172019-02-172019-03-132019-04-032019-04-142019-05-132019-06-202019-01-272019-03-062019-03-222019-04-102019-05-122019-05-212019-05-202019-05-232019-01-282019-01-292019-01-192019-02-252019-06-272019-04-102019-01-052


``` python
df.groupby(by = None, axis = 0, level = None, as_index: bool = True,
           sort: bool = True, group_keys: bool = True,
           squeeze: bool = <object object at 0x00000225ADF7E510>,
           observed: bool = False, dropna: bool = True
) -> 'DataFrameGroupBy'

```

# 5.0 Pivot table<a name="pivot"></a>

<div style="text-align: right"
     
[Voltar ao índice](#Contents)

``` python 
sinasc_3.pivot_table(values=None, index=None, columns=None, aggfunc='mean',
                    fill_value=None, margins=False, dropna=True, margins_name='All',
                    observed=False,
) -> 'DataFrame'

```

In [11]:
sinasc_1.pivot_table(index='ESCMAE',
                     aggfunc='mean')

TypeError: Could not convert 2019-03-272019-03-112019-03-282019-01-102019-01-242019-01-082019-01-142019-01-122019-01-232019-01-102019-01-152019-01-192019-01-242019-01-312019-02-152019-02-212019-02-112019-02-032019-02-142019-02-162019-02-162019-02-262019-02-232019-03-012019-03-012019-03-072019-03-152019-03-062019-03-032019-03-042019-03-182019-03-232019-03-262019-03-232019-03-262019-04-152019-04-082019-04-122019-04-232019-04-022019-04-082019-04-082019-04-122019-04-122019-04-192019-04-202019-04-222019-05-152019-05-082019-05-082019-05-102019-05-112019-05-272019-05-282019-06-172019-06-062019-06-172019-06-182019-06-292019-02-262019-03-272019-05-022019-05-092019-05-182019-01-022019-02-272019-06-202019-01-042019-02-182019-03-032019-04-112019-05-282019-05-312019-01-012019-02-132019-02-172019-02-172019-03-132019-04-032019-04-142019-05-132019-06-202019-01-272019-03-062019-03-222019-04-102019-05-122019-05-212019-05-202019-05-232019-01-282019-01-292019-01-192019-02-252019-06-272019-04-102019-01-052

# 6.0 Group by DTNASC e contagem da IDADEMAE <a name="dtnasc"></a>

<div style="text-align: right"
     
[Voltar ao índice](#Contents)

##  6.1 Em SQL <a name="sql"></a>

``` sql
SELECT DTNASC, count(IDADEMAE) 
FROM sinasc
GROUP BY DTNASC

```

## 6.2 Em excel <a name="excel"></a>

<img src="groupby_gif_2.gif" width="750" align="center">

## 6.3 Python com ``groupby()`` <a name="py_group"></a>

In [ ]:
sinasc_1.groupby('DTNASC')['IDADEMAE'].count().to_frame()

## 6.4 Python com ``pivot_table()`` <a name="py_pivot"></a>

In [ ]:
sinasc_1.pivot_table(values = 'IDADEMAE',index = 'DTNASC', aggfunc='count')

In [ ]:
sinasc_1.pivot_table(values = 'IDADEMAE',index = 'DTNASC', aggfunc='mean').plot(figsize=[18,5]);